In [1]:
from fastlane_bot import Bot, Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
from fastlane_bot.tools.analyzer import CPCAnalyzer
from fastlane_bot.tools.optimizer import CPCArbOptimizer
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCAnalyzer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCArbOptimizer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
from fastlane_bot.testing import *
import itertools as it
import collections as cl
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
ConstantProductCurve v2.10.2 (07/May/2023)
CPCAnalyzer v0.1 (06/May/2023)
CPCArbOptimizer v3.6 (06/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.1 [requirements >= 3.0 is met]


# Mainnet Server [A011]

In [2]:
bot     = Bot()
CCm     = bot.get_curves()
CA      = CPCAnalyzer(CCm)

In [3]:
pairs0  = CA.CC.pairs(standardize=False)
pairs   = CA.pairs()
pairsc  = CA.pairsc()
tokens  = CA.tokens()

In [4]:
assert CA.pairs()  == CCm.pairs(standardize=True)
assert CA.pairsc() == {c.pairo.primary for c in CCm if c.P("exchange")=="carbon_v1"}
assert CA.tokens() == CCm.tokens()

## Overall market

In [5]:
print(f"Total pairs:    {len(pairs0):4}")
print(f"Primary pairs:  {len(pairs):4}")
print(f"...carbon:      {len(pairsc):4}")
print(f"Tokens:         {len(CCm.tokens()):4}")
print(f"Curves:         {len(CCm):4}")

Total pairs:     665
Primary pairs:   636
...carbon:        25
Tokens:          512
Curves:          811


## By pair

### All pairs

In [6]:
cbp0 = {pair: [c for c in CCm.bypairs(pair)] for pair in CCm.pairs()} # curves by (primary) pair
nbp0 = {pair: len(cc) for pair,cc in cbp0.items()}
assert len(cbp0) == len(CCm.pairs())
assert set(cbp0) == CCm.pairs()

### Only those with >1 curves

In [7]:
cbp = {pair: cc for pair, cc in cbp0.items() if len(cc)>1}
nbp = {pair: len(cc) for pair,cc in cbp.items()}
print(f"Pairs with >1 curves:  {len(cbp)}")
print(f"Curves in those:       {sum(nbp.values())}")
print(f"Average curves/pair:   {sum(nbp.values())/len(cbp):.1f}")

Pairs with >1 curves:  110
Curves in those:       285
Average curves/pair:   2.6


### x=0 or y=0

In [8]:
xis0 = {c.cid: (c.x, c.y) for c in CCm if c.x==0}
yis0 = {c.cid: (c.x, c.y) for c in CCm if c.y==0}
assert len(xis0) == 0 # set loglevel debug to see removal of curves
assert len(yis0) == 0

### Prices

#### All

In [9]:
prices_da = {pair: 
             [(
                Pair.n(pair), pair, c.primaryp(), c.cid, c.cid[-8:], c.P("exchange"), c.tvl(tkn=pair.split("/")[0]),
                "x" if c.itm(cc) else "", c.buysell(verbose=False), c.buysell(verbose=True, withprice=True)
              ) for c in cc 
             ] 
             for pair, cc in cbp.items()
            }
#prices_da

#### Only for pairs that have at least on Carbon pair

In [10]:
prices_d = {pair: l for pair,l in prices_da.items() if pair in pairsc}
prices_l = list(it.chain(*prices_d.values()))

In [11]:
#prices_d

In [12]:
curves_by_pair = list(cl.Counter([r[1] for r in prices_l]).items())
curves_by_pair = sorted(curves_by_pair, key=lambda x: x[1], reverse=True)
curves_by_pair

[('WETH-6Cc2/USDC-eB48', 20),
 ('BNT-FF1C/WETH-6Cc2', 12),
 ('BNT-FF1C/vBNT-7f94', 10),
 ('USDT-1ec7/USDC-eB48', 8),
 ('WBTC-C599/WETH-6Cc2', 5),
 ('LINK-86CA/USDT-1ec7', 5),
 ('WBTC-C599/USDT-1ec7', 4),
 ('BNT-FF1C/USDC-eB48', 4),
 ('WETH-6Cc2/DAI-1d0F', 3),
 ('LINK-86CA/USDC-eB48', 3),
 ('PEPE-1933/WETH-6Cc2', 3),
 ('DAI-1d0F/USDC-eB48', 3),
 ('WETH-6Cc2/USDT-1ec7', 3),
 ('DAI-1d0F/USDT-1ec7', 3),
 ('LYXe-be6D/USDC-eB48', 2),
 ('WBTC-C599/USDC-eB48', 2),
 ('stETH-fE84/WETH-6Cc2', 2),
 ('ARB-4ad1/MATIC-eBB0', 2),
 ('0x0-1AD5/WETH-6Cc2', 2),
 ('rETH-6393/WETH-6Cc2', 2),
 ('TSUKA-69eD/USDC-eB48', 2)]

In [13]:
CODE = """
# #### {pairn}

pair = "{pair}"
pricedf.loc[Pair.n(pair)]

pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)
"""

In [14]:
# for pair, _ in curves_by_pair:
#     print(CODE.format(pairn=Pair.n(pair), pair=pair))

#### Dataframe

In [15]:
pricedf0 = pd.DataFrame(prices_l, columns="pair,pairf,price,cid,cid0,exchange,vl,itm,bs,bsv".split(","))
pricedf = pricedf0.drop(['cid', 'pairf'], axis=1).sort_values(by=["pair", "exchange", "cid0"])
pricedf = pricedf.set_index(["pair", "exchange", "cid0"])
pricedf

price            vl itm  bs  \
pair       exchange     cid0                                          
0x0/WETH   carbon_v1    132277-0     0.000013  7.757952e+03       b   
                        132277-1     0.000014  5.363000e+03       s   
ARB/MATIC  carbon_v1    806240-0     1.507045  1.276054e+01       s   
                        806240-1     1.428571  1.418060e+02       b   
BNT/USDC   bancor_v2    652          0.469913  1.492710e+06   x  bs   
...                                       ...           ...  ..  ..   
WETH/USDT  uniswap_v2   256       1918.786596  3.195897e+04      bs   
rETH/WETH  carbon_v1    903115-0     1.069000  1.870907e+00       b   
           sushiswap_v2 833          1.237861  3.116368e-04      bs   
stETH/WETH carbon_v1    422914-0     1.010101  2.031521e-03       s   
                        422914-1     0.990099  8.011450e-02       b   

                                                                    bsv  
pair       exchange     cid0                                             
0x0/WETH   carbon_v1    132277-0            buy-0x0 @ 0.00 WETH per 0x0  
                        132277-1           sell-0x0 @ 0.00 WETH per 0x0  
ARB/MATIC  carbon_v1    806240-0          sell-ARB @ 1.51 MATIC per ARB  
                        806240-1           buy-ARB @ 1.43 MATIC per ARB  
BNT/USDC   bancor_v2    652            buy-sell-BNT @ 0.47 USDC per BNT  
...                                                                 ...  
WETH/USDT  uniswap_v2   256       buy-sell-WETH @ 1918.79 USDT per WETH  
rETH/WETH  carbon_v1    903115-0          buy-rETH @ 1.07 WETH per rETH  
           sushiswap_v2 833          buy-sell-rETH @ 1.24 WETH per rETH  
stETH/WETH carbon_v1    422914-0       sell-stETH @ 1.01 WETH per stETH  
                        422914-1        buy-stETH @ 0.99 WETH per stETH  

[100 rows x 5 columns]

### Individual frames

#### WETH/USDC

In [16]:
pair = "WETH-6Cc2/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                          
carbon_v1    057285-1  2099.999790      0.006040       s   
             057292-0  1853.408818      0.003314       b   
             057292-1  2000.000000      0.016387       s   
             057296-0  1929.999824      0.461405   x   b   
             057296-1  1949.999818     10.000019       s   
             057299-1  1940.000000      0.026117       s   
             057306-0  1405.000140      3.558719       b   
             057315-1  2300.000000      0.487950       s   
             057331-0  1800.000000      5.555556       b   
             057334-0  1700.000170      0.029412       b   
             057334-1  1999.999800      0.040000       s   
             057337-0  1850.000000      1.081081       b   
             057339-0  1800.000000      0.000556       b   
             057343-1  1989.999801      1.000000       s   
             057353-0  1854.000185      4.234699       b   
             057353-1  2047.999795      4.000000       s   
sushiswap_v2 803       1918.328630  18000.709598      bs   
uniswap_v2   255       1917.166981  38437.683898      bs   
uniswap_v3   346       1915.241349    223.316190      bs   
             593       1910.636978     22.022255   x  bs   

                                                         bsv  
exchange     cid0                                             
carbon_v1    057285-1      sell-WETH @ 2100.00 USDC per WETH  
             057292-0       buy-WETH @ 1853.41 USDC per WETH  
             057292-1      sell-WETH @ 2000.00 USDC per WETH  
             057296-0       buy-WETH @ 1930.00 USDC per WETH  
             057296-1      sell-WETH @ 1950.00 USDC per WETH  
             057299-1      sell-WETH @ 1940.00 USDC per WETH  
             057306-0       buy-WETH @ 1405.00 USDC per WETH  
             057315-1      sell-WETH @ 2300.00 USDC per WETH  
             057331-0       buy-WETH @ 1800.00 USDC per WETH  
             057334-0       buy-WETH @ 1700.00 USDC per WETH  
             057334-1      sell-WETH @ 2000.00 USDC per WETH  
             057337-0       buy-WETH @ 1850.00 USDC per WETH  
             057339-0       buy-WETH @ 1800.00 USDC per WETH  
             057343-1      sell-WETH @ 1990.00 USDC per WETH  
             057353-0       buy-WETH @ 1854.00 USDC per WETH  
             057353-1      sell-WETH @ 2048.00 USDC per WETH  
sushiswap_v2 803       buy-sell-WETH @ 1918.33 USDC per WETH  
uniswap_v2   255       buy-sell-WETH @ 1917.17 USDC per WETH  
uniswap_v3   346       buy-sell-WETH @ 1915.24 USDC per WETH  
             593       buy-sell-WETH @ 1910.64 USDC per WETH

In [17]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0], params=dict(verbose=False, debug=False))
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,USDC-eB48
1701411834604692317316873037158841057296-0,0.461405,-8.905124e+02
255,9.065531,-1.737194e+04
593,-2.726753,5.216271e+03
803,6.973088,-1.336632e+04
346,-13.790266,2.641250e+04
AMMIn,16.500025,3.162877e+04
AMMOut,-16.517020,-3.162877e+04
TOTAL NET,-0.016995,-3.511086e-07


In [18]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,USDC-eB48
1701411834604692317316873037158841057296-0,4.614054e-01,-890.512360
255,9.066216e+00,-17373.255049
593,-2.726675e+00,5216.120112
803,6.973409e+00,-13366.933046
346,-1.377436e+01,26382.029063
AMMIn,1.650103e+01,31598.149176
AMMOut,-1.650103e+01,-31630.700454
TOTAL NET,9.686119e-11,-32.551278


#### BNT/WETH

In [19]:
pair = "BNT-FF1C/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange   cid0                                       
bancor_v2  623       0.000242  1.206183e+07   x  bs   
bancor_v3  704       0.000246  3.125669e+07   x  bs   
carbon_v1  326030-0  0.000253  5.000000e+02       s   
           326030-1  0.000200  2.500000e+02       b   
           326031-0  0.000200  7.499999e+02       b   
           326031-1  0.002000  5.000000e+02       s   
           326034-0  0.000200  3.500000e+02       b   
           326034-1  0.002100  2.000000e+02       s   
           326076-0  0.000253  7.905138e+02   x   b   
           326077-0  0.000250  3.654723e-01   x   b   
           326077-1  0.000258  7.936392e+03       s   
uniswap_v2 290       0.000243  2.137445e+05   x  bs   

                                                  bsv  
exchange   cid0                                        
bancor_v2  623       buy-sell-BNT @ 0.00 WETH per BNT  
bancor_v3  704       buy-sell-BNT @ 0.00 WETH per BNT  
carbon_v1  326030-0      sell-BNT @ 0.00 WETH per BNT  
           326030-1       buy-BNT @ 0.00 WETH per BNT  
           326031-0       buy-BNT @ 0.00 WETH per BNT  
           326031-1      sell-BNT @ 0.00 WETH per BNT  
           326034-0       buy-BNT @ 0.00 WETH per BNT  
           326034-1      sell-BNT @ 0.00 WETH per BNT  
           326076-0       buy-BNT @ 0.00 WETH per BNT  
           326077-0       buy-BNT @ 0.00 WETH per BNT  
           326077-1      sell-BNT @ 0.00 WETH per BNT  
uniswap_v2 290       buy-sell-BNT @ 0.00 WETH per BNT

In [20]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,BNT-FF1C
290,1.113021e-01,-456.798537
3743106036130323098097120681749450326076-0,-1.270395e-01,510.574438
623,7.487870e+00,-30756.718399
3743106036130323098097120681749450326077-0,-9.136815e-05,0.365472
704,-7.472041e+00,30475.749543
AMMIn,7.599172e+00,30986.689454
AMMOut,-7.599172e+00,-31213.516936
TOTAL NET,-1.745999e-10,-226.827482


In [21]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,BNT-FF1C
290,0.111031,-4.556911e+02
3743106036130323098097120681749450326076-0,-0.127118,5.108958e+02
623,7.472590,-3.069428e+04
3743106036130323098097120681749450326077-0,-0.000091,3.654725e-01
704,-7.511916,3.063871e+04
AMMIn,7.583621,3.114997e+04
AMMOut,-7.639126,-3.114997e+04
TOTAL NET,-0.055505,5.590118e-08


#### BNT/vBNT

In [22]:
pair = "BNT-FF1C/vBNT-7f94"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange  cid0                                       
bancor_v2 653       1.333016  5.350297e+04   x  bs   
bancor_v3 742       1.252518  2.466230e+06   x  bs   
carbon_v1 748950-0  0.940000  1.413879e+04       b   
          748965-1  0.909091  1.129751e+03       b   
          748966-1  1.000000  1.089256e+03       b   
          748976-0  1.331480  2.593859e+02       s   
          748976-1  1.111111  7.293739e+02       b   
          748977-0  1.428571  5.000000e+02       s   
          748977-1  1.250000  4.000000e+02       b   
          748990-1  0.952381  8.383963e+02       b   

                                                 bsv  
exchange  cid0                                        
bancor_v2 653       buy-sell-BNT @ 1.33 vBNT per BNT  
bancor_v3 742       buy-sell-BNT @ 1.25 vBNT per BNT  
carbon_v1 748950-0       buy-BNT @ 0.94 vBNT per BNT  
          748965-1       buy-BNT @ 0.91 vBNT per BNT  
          748966-1       buy-BNT @ 1.00 vBNT per BNT  
          748976-0      sell-BNT @ 1.33 vBNT per BNT  
          748976-1       buy-BNT @ 1.11 vBNT per BNT  
          748977-0      sell-BNT @ 1.43 vBNT per BNT  
          748977-1       buy-BNT @ 1.25 vBNT per BNT  
          748990-1       buy-BNT @ 0.95 vBNT per BNT

In [23]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,vBNT-7f94,BNT-FF1C
742,1.069551e+03,-853.329762
653,-1.069551e+03,827.163141
AMMIn,1.069551e+03,827.163141
AMMOut,-1.069551e+03,-853.329762
TOTAL NET,-1.964509e-10,-26.166621


In [24]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,vBNT-7f94,BNT-FF1C
742,1037.450295,-8.277359e+02
653,-1070.269176,8.277359e+02
AMMIn,1037.450295,8.277359e+02
AMMOut,-1070.269176,-8.277359e+02
TOTAL NET,-32.818881,-2.692104e-10


#### USDT/USDC

In [25]:
pair = "USDT-1ec7/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    634371-0  1.005025  5.010000e+01       s   
             634371-1  0.995025  5.040025e+01       b   
             634391-0  1.001001  5.050000e+02       s   
             634391-1  1.000690  4.946550e+02       b   
sushiswap_v2 805       0.993768  1.849228e+03      bs   
uniswap_v2   246       1.002118  2.369742e+07      bs   
uniswap_v3   357       1.002654  3.514249e+03      bs   
             486       1.001069  1.011262e+06      bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    634371-0      sell-USDT @ 1.01 USDC per USDT  
             634371-1       buy-USDT @ 1.00 USDC per USDT  
             634391-0      sell-USDT @ 1.00 USDC per USDT  
             634391-1       buy-USDT @ 1.00 USDC per USDT  
sushiswap_v2 805       buy-sell-USDT @ 0.99 USDC per USDT  
uniswap_v2   246       buy-sell-USDT @ 1.00 USDC per USDT  
uniswap_v3   357       buy-sell-USDT @ 1.00 USDC per USDT  
             486       buy-sell-USDT @ 1.00 USDC per USDT

In [26]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,USDC-eB48
357,927.516022,-9.292427e+02
246,6202.883776,-6.212769e+03
486,-7092.599827,7.100184e+03
805,-3.378126,3.369384e+00
1020847100762815390390123822295304634391-0,-38.419050,3.845883e+01
AMMIn,7130.399798,7.142012e+03
AMMOut,-7134.397002,-7.142012e+03
TOTAL NET,-3.997205,-2.563731e-07


In [27]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,USDC-eB48
357,927.516253,-929.242938
246,6202.886117,-6212.771617
486,-7088.605419,7096.185093
805,-3.378126,3.369384
1020847100762815390390123822295304634391-0,-38.418828,38.458604
AMMIn,7130.402371,7138.013080
AMMOut,-7130.402373,-7142.014554
TOTAL NET,-0.000003,-4.001474


#### WBTC/WETH

In [28]:
pair = "WBTC-C599/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price          vl itm  bs  \
exchange     cid0                                      
carbon_v1    709362-0  15.399750    0.133758       s   
             709362-1  14.285714    0.417087       b   
sushiswap_v2 804       15.099383  776.430469   x  bs   
uniswap_v3   466       15.097158    0.127246   x  bs   
             478       15.312285    0.000571   x  bs   

                                                       bsv  
exchange     cid0                                           
carbon_v1    709362-0      sell-WBTC @ 15.40 WETH per WBTC  
             709362-1       buy-WBTC @ 14.29 WETH per WBTC  
sushiswap_v2 804       buy-sell-WBTC @ 15.10 WETH per WBTC  
uniswap_v3   466       buy-sell-WBTC @ 15.10 WETH per WBTC  
             478       buy-sell-WBTC @ 15.31 WETH per WBTC

In [29]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,WBTC-C599
466,1.402761e-02,-0.000929
804,-5.809137e-03,0.000385
478,-8.218471e-03,0.000537
AMMIn,1.402761e-02,0.000921
AMMOut,-1.402761e-02,-0.000929
TOTAL NET,-7.038921e-10,-0.000008


In [30]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,WBTC-C599
466,0.014024,-9.288454e-04
804,-0.005920,3.920961e-04
478,-0.008218,5.367493e-04
AMMIn,0.014024,9.288454e-04
AMMOut,-0.014139,-9.288454e-04
TOTAL NET,-0.000115,-5.528378e-11


#### LINK/USDT

In [31]:
pair = "LINK-86CA/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price         vl itm  bs  \
exchange     cid0                                    
carbon_v1    960408-0  7.100000  26.783355   x   b   
             960408-1  7.700000  10.874600       s   
sushiswap_v2 791       7.123545  75.100134   x  bs   
uniswap_v2   171       7.328775  65.660409   x  bs   
uniswap_v3   549       6.976809   3.304874   x  bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    960408-0       buy-LINK @ 7.10 USDT per LINK  
             960408-1      sell-LINK @ 7.70 USDT per LINK  
sushiswap_v2 791       buy-sell-LINK @ 7.12 USDT per LINK  
uniswap_v2   171       buy-sell-LINK @ 7.33 USDT per LINK  
uniswap_v3   549       buy-sell-LINK @ 6.98 USDT per LINK

In [32]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LINK-86CA,USDT-1ec7
171,0.548893,-3.956563e+00
549,-2.045357,1.429659e+01
791,0.089456,-6.357306e-01
4763953136893138488487244504044754960408-0,1.367792,-9.704294e+00
AMMIn,2.006141,1.429659e+01
AMMOut,-2.045357,-1.429659e+01
TOTAL NET,-0.039216,-2.103206e-12


In [33]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LINK-86CA,USDT-1ec7
171,5.495607e-01,-3.961296
549,-2.045357e+00,14.296587
791,9.020908e-02,-0.641068
4763953136893138488487244504044754960408-0,1.405587e+00,-9.972246
AMMIn,2.045357e+00,14.296587
AMMOut,-2.045357e+00,-14.574610
TOTAL NET,7.105427e-14,-0.278023


#### WBTC/USDT

In [34]:
pair = "WBTC-C599/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price        vl itm  bs  \
exchange     cid0                                       
carbon_v1    920820-0  28980.881784  0.006836       b   
             920820-1  29500.000000  0.000065       s   
sushiswap_v2 814       29043.429150  0.031191      bs   
uniswap_v2   183       28831.097041  0.336923      bs   

                                                          bsv  
exchange     cid0                                              
carbon_v1    920820-0       buy-WBTC @ 28980.88 USDT per WBTC  
             920820-1      sell-WBTC @ 29500.00 USDT per WBTC  
sushiswap_v2 814       buy-sell-WBTC @ 29043.43 USDT per WBTC  
uniswap_v2   183       buy-sell-WBTC @ 28831.10 USDT per WBTC

In [35]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,WBTC-C599
183,7.247198e+00,-2.509929e-04
814,-9.853793e-01,3.400176e-05
9527906273786276976974489008089509920820-0,-6.261819e+00,2.163050e-04
AMMIn,7.247198e+00,2.503068e-04
AMMOut,-7.247198e+00,-2.509929e-04
TOTAL NET,-2.025644e-07,-6.861208e-07


In [36]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,WBTC-C599
183,7.238429,-2.506896e-04
814,-0.986194,3.402993e-05
9527906273786276976974489008089509920820-0,-6.272076,2.166597e-04
AMMIn,7.238429,2.506896e-04
AMMOut,-7.258270,-2.506896e-04
TOTAL NET,-0.019841,-6.381194e-12


#### BNT/USDC

In [37]:
pair = "BNT-FF1C/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange  cid0                                       
bancor_v2 652       0.469913  1.492710e+06   x  bs   
bancor_v3 720       0.465186  5.081971e+06   x  bs   
carbon_v1 480199-0  2.000000  2.910000e+01       s   
          480202-1  1.480041  4.247463e+04       s   

                                                 bsv  
exchange  cid0                                        
bancor_v2 652       buy-sell-BNT @ 0.47 USDC per BNT  
bancor_v3 720       buy-sell-BNT @ 0.47 USDC per BNT  
carbon_v1 480199-0      sell-BNT @ 2.00 USDC per BNT  
          480202-1      sell-BNT @ 1.48 USDC per BNT

In [38]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,USDC-eB48
652,2917.222349,-1.365503e+03
720,-2932.007880,1.365503e+03
AMMIn,2917.222349,1.365503e+03
AMMOut,-2932.007880,-1.365503e+03
TOTAL NET,-14.785530,-1.746230e-10


In [39]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,USDC-eB48
652,2.920592e+03,-1367.074614
720,-2.920592e+03,1360.180729
AMMIn,2.920592e+03,1360.180729
AMMOut,-2.920592e+03,-1367.074614
TOTAL NET,-5.820766e-10,-6.893884


#### WETH/DAI

In [40]:
pair = "WETH-6Cc2/DAI-1d0F"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,211457-1,1944.999806,0.001000,,s,sell-WETH @ 1945.00 DAI per WETH
sushiswap_v2,817,1923.592818,4184.139471,x,bs,buy-sell-WETH @ 1923.59 DAI per WETH
uniswap_v3,594,1902.663470,8.339070,x,bs,buy-sell-WETH @ 1902.66 DAI per WETH


In [41]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,DAI-1d0F,WETH-6Cc2
594,6.243703e+03,-3.268730
817,-6.243703e+03,3.250899
AMMIn,6.243703e+03,3.250899
AMMOut,-6.243703e+03,-3.268730
TOTAL NET,-2.188608e-08,-0.017831


In [42]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,DAI-1d0F,WETH-6Cc2
594,6233.978193,-3.263658e+00
817,-6268.171380,3.263658e+00
AMMIn,6233.978193,3.263658e+00
AMMOut,-6268.171380,-3.263658e+00
TOTAL NET,-34.193188,-1.239187e-11


#### DAI/USDT

In [43]:
pair = "DAI-1d0F/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price          vl itm  bs  \
exchange     cid0                                     
carbon_v1    268742-0  0.995000   20.100501       b   
             268742-1  1.005000   20.000000       s   
sushiswap_v2 795       0.994182  380.253948      bs   

                                                    bsv  
exchange     cid0                                        
carbon_v1    268742-0       buy-DAI @ 1.00 USDT per DAI  
             268742-1      sell-DAI @ 1.00 USDT per DAI  
sushiswap_v2 795       buy-sell-DAI @ 0.99 USDT per DAI

In [44]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [45]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### DAI/USDC

In [46]:
pair = "DAI-1d0F/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    845828-0  1.001001     30.000000       s   
             845828-1  0.999001     20.019998       b   
sushiswap_v2 839       1.003088  25646.214235      bs   

                                                    bsv  
exchange     cid0                                        
carbon_v1    845828-0      sell-DAI @ 1.00 USDC per DAI  
             845828-1       buy-DAI @ 1.00 USDC per DAI  
sushiswap_v2 839       buy-sell-DAI @ 1.00 USDC per DAI

In [47]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [48]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### WETH/USDT

In [49]:
pair = "WETH-6Cc2/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,691656-0,1900.000190,0.002632,,b,buy-WETH @ 1900.00 USDT per WETH
sushiswap_v2,840,1916.546534,10462.631094,,bs,buy-sell-WETH @ 1916.55 USDT per WETH
uniswap_v2,256,1918.786596,31958.967041,,bs,buy-sell-WETH @ 1918.79 USDT per WETH


In [50]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,USDT-1ec7
256,2.300478,-4.413491e+03
840,-2.301822,4.413491e+03
AMMIn,2.300478,4.413491e+03
AMMOut,-2.301822,-4.413491e+03
TOTAL NET,-0.001344,1.862645e-08


In [51]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,USDT-1ec7
256,2.301491e+00,-4415.433842
840,-2.301491e+00,4412.855720
AMMIn,2.301491e+00,4412.855720
AMMOut,-2.301491e+00,-4415.433842
TOTAL NET,2.728484e-12,-2.578122


#### LINK/USDC

In [52]:
pair = "LINK-86CA/USDC-eB48"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,497903-1,7.750000,342.883761,,s,sell-LINK @ 7.75 USDC per LINK
sushiswap_v2,792,6.935941,3.543424,x,bs,buy-sell-LINK @ 6.94 USDC per LINK
uniswap_v2,176,6.794102,183.739077,x,bs,buy-sell-LINK @ 6.79 USDC per LINK


In [53]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,LINK-86CA
176,1.238839e-01,-0.018230
792,-1.238839e-01,0.018043
AMMIn,1.238839e-01,0.018043
AMMOut,-1.238839e-01,-0.018230
TOTAL NET,-1.474376e-13,-0.000187


In [54]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,LINK-86CA
176,0.122635,-1.804664e-02
792,-0.123908,1.804664e-02
AMMIn,0.122635,1.804664e-02
AMMOut,-0.123908,-1.804664e-02
TOTAL NET,-0.001273,1.021405e-14


#### PEPE/WETH

In [55]:
pair = "PEPE-1933/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price            vl itm bs  \
exchange  cid0                                          
carbon_v1 440620-0  3.500000e-07  1.454145e+04      b   
          440620-1  4.000000e-07  7.130133e+06      s   
          440621-1  4.500000e-07  1.315789e+06      s   

                                               bsv  
exchange  cid0                                      
carbon_v1 440620-0   buy-PEPE @ 0.00 WETH per PEPE  
          440620-1  sell-PEPE @ 0.00 WETH per PEPE  
          440621-1  sell-PEPE @ 0.00 WETH per PEPE

In [56]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [57]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### stETH/WETH

In [58]:
pair = "stETH-fE84/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price        vl itm bs  \
exchange  cid0                                  
carbon_v1 422914-0  1.010101  0.002032      s   
          422914-1  0.990099  0.080114      b   

                                                 bsv  
exchange  cid0                                        
carbon_v1 422914-0  sell-stETH @ 1.01 WETH per stETH  
          422914-1   buy-stETH @ 0.99 WETH per stETH

In [59]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [60]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### rETH/WETH

In [61]:
pair = "rETH-6393/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,903115-0,1.069000,1.870907,,b,buy-rETH @ 1.07 WETH per rETH
sushiswap_v2,833,1.237861,0.000312,,bs,buy-sell-rETH @ 1.24 WETH per rETH


In [62]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,rETH-6393
833,0.0,0.0
AMMIn,0.0,0.0
AMMOut,0.0,0.0
TOTAL NET,0.0,0.0


In [63]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WETH-6Cc2,rETH-6393
833,0.0,0.0
AMMIn,0.0,0.0
AMMOut,0.0,0.0
TOTAL NET,0.0,0.0


#### ARB/MATIC

In [64]:
pair = "ARB-4ad1/MATIC-eBB0"
pricedf.loc[Pair.n(pair)]

price          vl itm bs                            bsv
exchange  cid0                                                                
carbon_v1 806240-0  1.507045   12.760538      s  sell-ARB @ 1.51 MATIC per ARB
          806240-1  1.428571  141.806023      b   buy-ARB @ 1.43 MATIC per ARB

In [65]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [66]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### 0x0/WETH

In [67]:
pair = "0x0-1AD5/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price           vl itm bs                           bsv
exchange  cid0                                                                
carbon_v1 132277-0  0.000013  7757.951901      b   buy-0x0 @ 0.00 WETH per 0x0
          132277-1  0.000014  5363.000000      s  sell-0x0 @ 0.00 WETH per 0x0

In [68]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [69]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### TSUKA/USDC

In [70]:
pair = "TSUKA-69eD/USDC-eB48"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,017697-1,0.120000,4.656734e+04,,s,sell-TSUKA @ 0.12 USDC per TSUKA
uniswap_v2,254,0.032085,4.336406e+07,,bs,buy-sell-TSUKA @ 0.03 USDC per TSUKA


In [71]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,TSUKA-69eD,USDC-eB48
254,0.0,0.0
AMMIn,0.0,0.0
AMMOut,0.0,0.0
TOTAL NET,0.0,0.0


In [72]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,TSUKA-69eD,USDC-eB48
254,0.0,0.0
AMMIn,0.0,0.0
AMMOut,0.0,0.0
TOTAL NET,0.0,0.0


#### WBTC/USDC

In [73]:
pair = "WBTC-C599/USDC-eB48"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,537493-0,28000.000000,0.045365,,b,buy-WBTC @ 28000.00 USDC per WBTC
uniswap_v2,332,28931.165371,6.015072,,bs,buy-sell-WBTC @ 28931.17 USDC per WBTC


In [74]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,WBTC-C599
332,4.365575e-11,-1.332268e-15
AMMIn,4.365575e-11,0.000000e+00
AMMOut,0.000000e+00,-1.332268e-15
TOTAL NET,4.365575e-11,-1.332268e-15


In [75]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,WBTC-C599
332,4.365575e-11,-1.332268e-15
AMMIn,4.365575e-11,0.000000e+00
AMMOut,0.000000e+00,-1.332268e-15
TOTAL NET,4.365575e-11,-1.332268e-15


#### LYXe/USDC

In [76]:
pair = "LYXe-be6D/USDC-eB48"
pricedf.loc[Pair.n(pair)]

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,652071-1,15.999998,7503.700799,,s,sell-LYXe @ 16.00 USDC per LYXe
uniswap_v3,558,14.391360,1.085336,,bs,buy-sell-LYXe @ 14.39 USDC per LYXe


In [77]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LYXe-be6D,USDC-eB48
558,0.0,0.0
AMMIn,0.0,0.0
AMMOut,0.0,0.0
TOTAL NET,0.0,0.0


In [78]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,LYXe-be6D
558,6.821210e-13,-5.684342e-14
AMMIn,6.821210e-13,0.000000e+00
AMMOut,0.000000e+00,-5.684342e-14
TOTAL NET,6.821210e-13,-5.684342e-14
